In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import py7zlib
from io import StringIO
import dask.bag as db
from dask.distributed import Client
client = Client()
# check status at: http://localhost:8787/status

In [38]:
data_dir = "data/ares"
dump_dir = f"{data_dir}/2018-01-10"

## Sources of information

In [3]:
vr_full_name = pd.read_csv(f"{data_dir}/ares_ciselnik_VR.csv", sep=';', header=None).set_index(0)[1]
vr_full_name

0
NR                              Nadační rejstřík
OR                             Obchodní rejstřík
ROPS     Rejstřík obecně prospěšných společností
RSVJ    Rejstřík společenství vlastníků jednotek
RU                               Rejstřík ústavů
SR                             Spolkový rejstřík
Name: 1, dtype: object

## Available ICOs

In [43]:
def read_7z(data_dir, base_filename, read_func):
    
    with open(f"{data_dir}/{base_filename}.7z", 'rb') as file:
        archive = py7zlib.Archive7z(file)

        csv = StringIO(archive.getmember(base_filename).read().decode())

        return read_func(csv)
    
pd_read_csv_ares = lambda csv, col_names: pd.read_csv(csv, sep=';', header=None, names=col_names)
pd_read_csv_ares_7z = lambda d, fn, col_names: read_7z(d, fn, lambda csv: pd_read_csv_ares(csv, col_names)) 

In [47]:
# Obsahem je seznam IČO s datem posledního zpracování v IS ARES.
ico_all = pd_read_csv_ares_7z(data_dir, "ares_seznamIC_VR.csv", ['ico', 'date_processed'])
ico_all.head()

,ico,date_processed
0,108,24.10.2015
1,124,24.10.2015
2,205,12.01.2017
3,248,24.10.2015
4,256,24.10.2015


In [48]:
# Obsahem je seznam IČO, pro které byl vytvořen Balík.
ico_dump = pd_read_csv_ares_7z(dump_dir, "ares_seznamIC_VR_balik.csv", ['ico'])
ico_dump.head()

,ico
0,108
1,124
2,205
3,248
4,256


In [51]:
len(set(ico_dump['ico']) & set(ico_all['ico']))

973595

In [53]:
len(set(ico_dump['ico']) - set(ico_all['ico']))

1660

In [54]:
len(set(ico_all['ico']) - set(ico_dump['ico']))

0

## The main dataset